# Exploring trading pairs

An example how to filter and plot candles for pair data

We will

* Download pair and exchange map ("the trading universe")

* Filter out SushiSwap pairs

* Download 4 hour candle data for all pairs

* Display top 10 pairs by their 30d volume, nominated in the USD

First let's import libraries and initialise our dataset client.

In [2]:
from tradingstrategy.client import Client
client = await Client.create_jupyter_lite_client(api_key=globals().get("api_key",None))
if client==None:
    api_key=await input("Please enter an api key then re-run the notebook")


No existing Trading Strategy configuration found in /home/pyodide/.tradingstrategy/settings.json. Making config from keyword parameters.
Testing out API key: secret-token:tradingstra
The server replied accepted our API key and sent the following greetings:
Server version: 0.1
Message of the day: Han shot first
The API key setup complete.


Get the map of exchanges and pairs we are working on

In [3]:
from tradingstrategy.exchange import ExchangeUniverse
from pyarrow import Table

# Exchange map data is so small it does not need any decompression
exchange_universe: ExchangeUniverse = client.fetch_exchange_universe()

# Decompress the pair dataset to Python map
columnar_pair_table: Table = client.fetch_pair_universe()

print(f"Total pairs {len(columnar_pair_table)}, total exchanges {len(exchange_universe.exchanges)}")

Missing HTTP response content-length header for download https://tradingstrategy.ai/api/exchange-universe, headers are ItemsView({'content-type': 'application/json'})
(Unknown total file size): 1.60MB [00:00, 9.94MB/s]
100%|##########| 13.2M/13.2M [00:00<00:00, 38.0MB/s]


Total pairs 103904, total exchanges 2939


## Filtering pairs trading on Sushiswap

We are only interested in SushiSwap, the mosh pit of a degens, so filter pairs that are on this exchange only.
We specifically pick Sushiswap on Ethereum mainnet,
as it is the oldest and most active for all Sushiswap instances across EVM blockchains like Polygon, BSC, Fantom, etc.

Also we do not yet have multichain data on the dataset server, when I am writing this tutorial so...

In [4]:
import pandas as pd
from tradingstrategy.chain import ChainId

# Convert PyArrow table to Pandas format to continue working on it
all_pairs_dataframe = columnar_pair_table.to_pandas()

# Filter down to pairs that only trade on Sushiswap
sushi_swap = exchange_universe.get_by_chain_and_slug(ChainId.ethereum, "sushi")
sushi_pairs: pd.DataFrame = all_pairs_dataframe.loc[all_pairs_dataframe['exchange_id'] == sushi_swap.exchange_id]

print(f"Sushi on Ethereum has {len(sushi_pairs)} pairs")

Sushi on Ethereum has 741 pairs


## Calculating combined buy and sell volume

Now let's get pairs sorted out by 30d buy and sell volume

In [5]:
# Avoid doing set operations on a Pandas read-only view
# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
sushi_pairs = sushi_pairs.copy()
total_volume_30d = sushi_pairs["buy_volume_30d"] + sushi_pairs["sell_volume_30d"]
sushi_pairs["total_30d_volume"] = total_volume_30d
# Sort by highest volume firs   t
sushi_pairs = sushi_pairs.sort_values("total_30d_volume", ascending=False)

print("total_30d_volume column sum added")

total_30d_volume column sum added


## Output table for top-10 pairs

Then let's see the top 10 pairs we got and choose colums to display.

In [7]:
# See Pandas official documentation for table visualisation
# https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

import jinja2

output = sushi_pairs[["base_token_symbol", "quote_token_symbol", "total_30d_volume"]].copy()

output.style.format({
  'total_30d_volume': lambda val: f'${val:,.2f}',
})

output = output.rename(columns={
    'base_token_symbol': 'Base token',
    'quote_token_symbol': 'Quote token',
    'total_30d_volume': 'Volume 30d USD'})

top_10 = output.head(10)

top_10.style.format({
  'Volume 30d USD': lambda val: f'${val:,.2f}',
}).hide_index()

<ipython-input-7-3a2b202ca69e>:21: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  }).hide_index()


Base token,Quote token,Volume 30d USD
WETH,USDC,"$684,643,584.00"
LUNA,WETH,"$425,134,784.00"
UST,WETH,"$329,412,736.00"
WETH,USDT,"$256,029,712.00"
SYN,WETH,"$216,412,608.00"
WETH,DAI,"$151,029,840.00"
WETH,UST,"$128,074,528.00"
LDO,WETH,"$115,634,976.00"
SUSHI,WETH,"$115,124,440.00"
JPEG,WETH,"$75,895,488.00"
